In [1]:
import sys
sys.path.append("../src")

import requests

from tonpy import LiteClient, Cell, get_block_info, BlockId, BlockIdExt, \
        Address, Emulator, begin_cell, StackEntry, VmDict, CellSlice

from tonpy.utils.shard_account import shard_is_ancestor, shard_child
from tonpy.libs.python_ton import Bits256
from tonpy.autogen.block import Account, Transaction, Block, BlockInfo, BlockExtra, MessageAny, AccountState

In [2]:
# Example hashes:
#   27 txs in 1 block: 184C0C0EF74E279E63B363FCC81168AEB4C3769E4991FAF6982A5E232EE88541
#   After merge: 7A074921055E014CE6B5E7C929B12D9789F5419E771F1BD98C379A57DA15064A
#   TickTock: F8EAC78DA7005B67B4A473CB77CF90ACD4A31B883E0E6C2806ADB4C0D2B3E616

In [3]:
tx_hash = "745BA37BA8E0C62371E04B815ABDE4DB8802272336708567FFD0D96A54418422"

In [4]:
def get_tx_lt(tx_hash):
    query = '''query{transactions(hash: "%s"){lt,address,workchain,shard}}''' % tx_hash

    response = requests.post("https://dton.io/graphql", json={'query': query})
    
    data = response.json()['data']['transactions'][0]
    return int(data['lt']), int(data['workchain']), str(data['address']), str(data['shard'])

In [5]:
# You can not trust this data from dton.io, but IF it's correct it'll be prooved by `get_transactions` later
tx_lt, wc, address, shard = get_tx_lt(tx_hash)
block_lt = tx_lt - tx_lt % 1000000

tx_account = Address(f"{wc}:{address}")

In [6]:
# Get random LiteServer
lc = LiteClient(mode='roundrobin', threads=1)

# When you get transaction - you automatically proof validness by tx_hash
transactions = lc.get_transactions(tx_account, tx_lt, tx_hash, 16).transactions

# We need to get all TXs in this block
whitelist_txs = [transactions[0]]
get_state_from = None
cur_index = 0

if len(transactions) > 1:
    done = transactions[0].blkid != transactions[1].blkid
else:
    done = True
    
while not done:
    cur_index += 1
    if cur_index > len(transactions) - 1:
        # Get new 16txs to load
        transactions.extend(lc.get_transactions(tx_account, 
                                          whitelist_txs[-1].prev_trans_lt, 
                                          whitelist_txs[-1].prev_trans_hash, 16).transactions)
    if cur_index > len(transactions):
        break
    
    if whitelist_txs[-1].blkid == transactions[cur_index].blkid:
        whitelist_txs.append(transactions[cur_index])
    else:
        done = True
        

In [7]:
# If there >1 TX in block - we need to emulate all of them before our one
print("To emulate, before our TX: ", len(whitelist_txs) - 1)

whitelist_txs = list(reversed(whitelist_txs))

To emulate, before our TX:  0


In [8]:
# Get block with prev state
# WARNING: this block is not proofed
state_block = whitelist_txs[0].blkid
current_block = lc.get_block_header(state_block)
print("Current block: ", current_block.blk_id, "\n")

# We need to get master block, so we can download libs and config
if wc != -1:
    # tlb python autogen
    block = Block().cell_unpack(current_block.virt_blk_root)
    block_info = BlockInfo().cell_unpack(block.info, True)
    
    # get master block id from WC
    master = BlockIdExt(id_=BlockId(-1, 0x8000000000000000, block_info.master_ref.master.seq_no), 
          file_hash=int(block_info.master_ref.master.file_hash, 2), 
          root_hash=int(block_info.master_ref.master.root_hash, 2))
else:
    master = current_block.blk_id

print("Master block: ", master, "\n")


Current block:  (-1,8000000000000000,36319299):6F3066DA91587118B68D24A8CEF170632AD6613F65B0E47D9B73E4098609C19B:F54013BADF6D0658A7472BE5B1908B546F71F60A9424E95EE794735ECBEFC38F 

Master block:  (-1,8000000000000000,36319299):6F3066DA91587118B68D24A8CEF170632AD6613F65B0E47D9B73E4098609C19B:F54013BADF6D0658A7472BE5B1908B546F71F60A9424E95EE794735ECBEFC38F 



In [9]:
# Need to get rand_seed for block and get prev block with account state
current_full_block = lc.get_block(current_block.blk_id)

# It's stored in blockExtra
block = Block().cell_unpack(current_full_block)
block_info = BlockInfo().cell_unpack(block.info, True)
block_extra = BlockExtra().cell_unpack(block.extra, False)

rand_seed = int(block_extra.rand_seed, 2)

In [10]:
# To get prev block with account state - need to calculate correct shard (with account) if block is after merge
# If not after merge - just get prev seqno
if block_info.after_merge:    
    left = block_info.prev_ref.prev1
    left_shard = BlockIdExt(BlockId(wc, shard_child(current_block.blk_id.id.shard, True), left.seq_no),
                           root_hash=int(left.root_hash,2), file_hash=int(left.file_hash, 2))
    
    right = block_info.prev_ref.prev2
    right_shard = BlockIdExt(BlockId(wc, shard_child(current_block.blk_id.id.shard, False), right.seq_no),
                           root_hash=int(right.root_hash,2), file_hash=int(right.file_hash, 2))
    
    if shard_is_ancestor(left_shard.id.shard, tx_account.shard_prefix(60)):
        state_block = left_shard
    else:
        state_block = right_shard
else:
    current = block_info.prev_ref.prev
    state_block = BlockIdExt(BlockId(wc, current_block.blk_id.id.shard, current.seq_no),
                           root_hash=int(current.root_hash,2), file_hash=int(current.file_hash, 2))

In [11]:
print("State block: ", state_block, "\n")

State block:  (-1,8000000000000000,36319298):75887F59A905822745DBA438E59B6F47DDA0F1E5DEC3DB486867582E16C78EF2:D571A70FA71FDA1E1F24EB97781DD3077B9C61C6956C8A41DADB40EF0F4DE50B 



In [12]:
# Load account state from prev block
account_state = lc.get_account_state(f"{wc}:{address}", state_block)

if not account_state.root.is_null():
    # Convert answer to AccountShardState
    account_state = begin_cell() \
        .store_ref(account_state.root) \
        .store_uint(int(account_state.last_trans_hash, 16), 256) \
        .store_uint(account_state.last_trans_lt, 64).end_cell()
else:
     account_state = begin_cell() \
        .store_ref(begin_cell().store_uint(0, 1).end_cell()) \
        .store_uint(int(account_state.last_trans_hash, 16), 256) \
        .store_uint(account_state.last_trans_lt, 64) \
        .end_cell()

In [13]:
# Load config of prev key block of MC block where TX was
# WARNING: this key block have no proof
key_block, config = lc.get_config_all(master, from_not_trusted_keyblock=True)

In [14]:
print("Latest key block: ", key_block)

Latest key block:  (-1,8000000000000000,36317813):E3A91013932F2A5CD00912831E818007FBB72216F70070B600D5B890A0E502A4:02391552B7D91167FDFBF3E4F26D26FB7A320173A9B2EFEEE3FD9AC2B74352BF


In [15]:
# for emulation you need to get 16 latest mc blocks + 1 key block
old_mc_blocks = []
cur_seqno = master.id.seqno

for i in range(16):
    cur_seqno -= 1
    old_mc_blocks.append(lc.lookup_block(BlockId(-1, 0x8000000000000000, cur_seqno)).blk_id)

In [16]:
prev_block_data = [
    [i.to_data() for i in old_mc_blocks],
    key_block.to_data()
]

In [17]:
# Here we need to get all public libraries (todo: rewrite on new LS API if it'll be published)
# Alternative: deep_library_search 
def get_public_libs(block_hash, wc, seqno, shard):
    query = '''query{blocks(workchain: %s, seqno: %s, shard: %s, root_hash: "%s"){libs_hash}}''' % (wc, seqno, shard, block_hash)

    response = requests.post("https://dton.io/graphql", json={'query': query})
    
    data = response.json()['data']['blocks'][0]
    return data['libs_hash']

In [18]:
public_libs = get_public_libs(master.root_hash, master.id.workchain, master.id.seqno, master.id.shard)

print("Found: ", len(public_libs), " public libs")

libs_data = lc.get_libraries(public_libs)

Found:  40  public libs


In [19]:
account_state.get_hash()

'E999765C75C228B6A87A944A5060A3BD60C06B67A4572C8BD5C0D17DCEC00F62'

In [20]:
gg = lc.list_block_transactions_ext(master, 1)

In [21]:
for i in gg.transactions:
    tt = Transaction().cell_unpack(i)
    print(i.get_hash(), tt.dump()['account_addr'], tt.dump()['lt'])

745BA37BA8E0C62371E04B815ABDE4DB8802272336708567FFD0D96A54418422 3333333333333333333333333333333333333333333333333333333333333333 44856566000001


# External emulator

In [22]:
from tonpy.tvm.not_native.emulator_extern import EmulatorExtern

In [23]:
em = EmulatorExtern("/Users/tvorogme/projects/ton_original/build/emulator/libemulator.dylib", config)

em.set_rand_seed(rand_seed)
em.set_prev_blocks_info(prev_block_data)
em.set_libs(libs_data)

True

In [24]:
for tx in whitelist_txs:
    # Get current transaction LT & now
    current_tx = tx.transaction

    tx_tlb = Transaction()
    tx_tlb = tx_tlb.cell_unpack(current_tx, True)

    lt = tx_tlb.lt
    now = tx_tlb.now
    
    # Get current trasaction in_msg
    current_tx_cs = current_tx.begin_parse()
    tmp = current_tx_cs.load_ref(as_cs=True)

    if tmp.load_bool():
        in_msg = tmp.load_ref()
    else:
        # TICK TOCK
        in_msg = None

    if in_msg is None:
        success = em.emulate_tick_tock_transaction(
                    account_state,
                    tx_tlb.description.is_tock,
                    now,
                    lt
                )
    else:
        # Emulate
        success = em.emulate_transaction(
            account_state,
            in_msg,
            now,
            lt
        )
        
    assert success
    
    # Emulation transaction equal current transaction
    assert em.transaction.get_hash() == current_tx.get_hash()
    print(f"Emulate {current_tx.get_hash()} success")
    
    # Update account state, go to next transaction
    account_state = em.account.to_cell()

Emulate 745BA37BA8E0C62371E04B815ABDE4DB8802272336708567FFD0D96A54418422 success


In [25]:
c = Cell('te6ccuECBgEAATMAAMIAyAFOAWQCBgJmA7d1VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVAAAoItl/VcMMivqhK5MSly5gPjiO4BFJVoX5gNUtc524jnVd8y7JMQAAKCLZcBODZa+i5QABSAL1wGCAECAwABIACCch5AQcPbMNY8a4EWCvgTUKfHPliNw79s3R4rLQpkIVY1Ov5gYoeQikAIWdRP5BWWK+AN+0hyiH5zE7px8Qr7EOUCDTAxAF64DCQEBQCeQTZsPQkAAAAAAAAAAAAuAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABbwAAAAAAAAAAAAAAAAS1FLaRJ5QuM990nhh8UYSKv4bVGu4tw/IIW8MYUE5+OBB3WmFM=')



[ 1][t 1][1708956733.489479065][adnl-ext-connection.cpp:129][!outconn]	Client got error [PosixError : Connection reset by peer : 54 : Read from [fd:81] has failed]
[ 1][t 1][1708956733.496727228][adnl-ext-connection.cpp:129][!outconn]	Client got error [PosixError : Connection reset by peer : 54 : Read from [fd:81] has failed]
[ 1][t 1][1708956734.499046087][adnl-ext-connection.cpp:129][!outconn]	Client got error [PosixError : Connection reset by peer : 54 : Read from [fd:81] has failed]
[ 1][t 1][1708956735.593051910][adnl-ext-connection.cpp:129][!outconn]	Client got error [PosixError : Connection reset by peer : 54 : Read from [fd:81] has failed]
[ 1][t 1][1708956736.504400969][adnl-ext-connection.cpp:129][!outconn]	Client got error [PosixError : Connection reset by peer : 54 : Read from [fd:81] has failed]
[ 1][t 1][1708956737.502191305][adnl-ext-connection.cpp:129][!outconn]	Client got error [PosixError : Connection reset by peer : 54 : Read from [fd:81] has failed]
[ 1][t 1][170895

In [ ]:
c.get_hash()

In [ ]:
c.dump_as_tlb("Transaction")